# No bonds scenario

## Init locator

Instantiates all Lido contacts

In [1]:
from lido_sandbox.locator import Locator

locator = Locator()
_, lido = locator.lido
_, router = locator.staking_router

## Init Lido

Initializes the Lido contract and put [the stone in the elevator](https://img2.reactor.cc/pics/post/%D0%B2%D1%81%D1%91-%D0%BF%D0%BB%D0%BE%D1%85%D0%BE-%D1%84%D1%8D%D0%BD%D0%B4%D0%BE%D0%BC%D1%8B-%D0%BB%D0%B8%D1%84%D1%82-7197897.jpeg)

In [2]:
lido.balance = 1 * 10**18
lido.initialize()

## Submits from users

In [3]:
lido.submit('user 1', 64 * 10**18)
lido.submit('user 2', 64 * 10**18)

None

## Init staking modules

Adds 2 staking modules to the staking router and adds some operators and keys

In [4]:
router.add_staking_module(name='curated', staking_module_address='curated module', target_share=10000, staking_module_fee=500, treasury_fee=500)
router.add_staking_module(name='community', staking_module_address='community module', target_share=10000, staking_module_fee=700, treasury_fee=300)

operators_per_module = 5
submitted_keys_per_operator = 10

### Curated module

In [5]:
curated_module = router.get_staking_module_instance('curated module')
for no_index in range(operators_per_module):
    curated_module.add_node_operator('curated operator %d' % (no_index), 'curated operator %d' % (no_index))
    curated_module.add_signing_keys(
        no_index, 
        [["curated key %d.%d" % (no_index, key_index), "curated signature %d.%d" % (no_index, key_index)] for key_index in range(submitted_keys_per_operator)]
    )
    curated_module.set_node_operator_staking_limit(no_index, submitted_keys_per_operator)

### Community module

In [6]:
community_module = router.get_staking_module_instance('community module')
for no_index in range(operators_per_module):
    community_module.add_node_operator('community operator %d' % (no_index), 'community operator %d' % (no_index))
    community_module.add_signing_keys(
        no_index, 
        [["community key %d.%d" % (no_index, key_index), "community signature %d.%d" % (no_index, key_index)] for key_index in range(submitted_keys_per_operator)]
    )
    community_module.set_node_operator_staking_limit(no_index, submitted_keys_per_operator)

## Deposit

Performs deposits to the keys

In [7]:
lido.deposit(max_deposits_count=100, staking_module_id=1)
lido.deposit(max_deposits_count=100, staking_module_id=2)

## Lido statistics

In [8]:
from lido_sandbox.helpers import print_table, wei_to_eth

deposited_validators, cl_validators, cl_balance = lido.get_beacon_stat()

print_table(
    [[
        wei_to_eth(lido.total_supply()),
        wei_to_eth(lido.get_total_shares()),
        wei_to_eth(lido.get_buffered_ether()),
        wei_to_eth(cl_balance),
        cl_validators,
        deposited_validators,
    ]],
    [
        'total supply',
        'total shares',
        'buffered',
        'cl balance',
        'cl vals',
        'deposited vals'
    ]
)

total supply,total shares,buffered,cl balance,cl vals,deposited vals
129.0,129.0,1.0,0.0,0,4


## Stakers

In [9]:
from lido_sandbox.helpers import print_table

print_table(
    [[key, wei_to_eth(value), wei_to_eth(lido.get_pooled_eth_by_shares(value))] for key, value in lido._shares.items()],
    ['address', 'shares', 'steth']
)

ModuleNotFoundError: No module named 'helpers'

## Oracle report

Reports deposited validators and some rewards

In [ ]:
from time import time
current_timestamp = int(time())

lido.handle_oracle_report(
    report_timestamp = current_timestamp,
    time_elapsed = current_timestamp - lido._last_report_timestamp,
    cl_validators = lido._deposited_validators,
    cl_balance = lido._deposited_validators * lido.DEPOSIT_SIZE,
    withdrawal_vault_balance = 2 * 10 ** 18,
    el_rewards_vault_balance = 2 * 10 ** 18,
    shares_requested_to_burn = 0,
    withdrawal_finalization_batches = [],
    simulated_share_rate = 0
)
None

## Stakers

In [ ]:
print_table(
    [[key, wei_to_eth(value), wei_to_eth(lido.get_pooled_eth_by_shares(value))] for key, value in lido._shares.items()],
    ['address', 'shares', 'steth']
)